# Gerardo de Miguel González

## M1964 Assignment: Building a weather generator

### Python implementation

### Description

Weather generators are computational models which generate (daily) synthetic series of random values simulating the distribution of a particular meteorological variable. Precipitation is the most popular example due to its mixed character: discrete (ocurrence: dry or wet) and continuous (rain amount for wet days). The discrete part is typically characterized by the frequency of wet/dry days (say, e.g. 0.53/0.47 in Santander), whereas the continuous is typically characterized by an exponential distribution defined by a rate parameter.

The main control structures (conditionals and loops) will be used and applied to build a simple weather generator and create artificial precipitation amount series. Some functions which simulate (pseudo)random numbers from different families of distributions (uniform and exponential) will also be used. This is a key task for several computational statistics problems.

### How does a precipitation time series look like?

In order to work with real data, we will use two files with daily information for two weather stations: Santander CMT (RR_STAID003922.txt) and Santander/Parayas (RR_STAID003923.txt). The former is located in the Santander regional center (close to el Faro de Cabo Mayor) and the latter is located in Santander airport. 

#### ::GMG::Task 1

I will open Santander CMT dataset (RR_STAID003922.txt) and see the structure. it has some lines with information (to be skipped) and then a header followed by number in columns (STAID: the code of the station; SOUI: the international code of the station; DATA: the day; RR: precipitation value; Q_RR: a quality control flat, 0: OK; 9: missing data.

::GMG::First 20 line metadata exceprt ...

> EUROPEAN CLIMATE ASSESSMENT & DATASET (ECA&D), file created on: 10-11-2017
>
> THESE DATA CAN BE USED FREELY FOR NON-COMMERCIAL RESEARCH PROVIDED THAT THE FOLLOWING SOURCE IS ACKNOWLEDGED: 
>
> Klein Tank, A.M.G. and Coauthors, 2002. Daily dataset of 20th-century surface
air temperature and precipitation series for the European Climate Assessment.
Int. J. of Climatol., 22, 1441-1453.
>
> Data and metadata available at http://www.ecad.eu
>
> FILE FORMAT (MISSING VALUE CODE = -9999):
>
> 01-06 STAID: Station identifier
> 08-13 SOUID: Source identifier
> 15-22 DATE : Date YYYYMMDD
> 24-28 RR   : Precipitation amount in 0.1 mm
> 30-34 Q_RR : quality code for RR (0='valid'; 1='suspect'; 9='missing')
>
> This is the blended series of station SANTANDER CMT, SPAIN (STAID: 3922)
>
> Blended and updated with sources:19972 19981 908023 
>
> See files sources.txt and stations.txt for more info.

#### ::GMG::Task 2

I will load the Santander CMT dataset and plot the first year available of precipitation data (RR column, year 1924 it seems ...). As the precipitation data is in 0.1 mm units so the actual mm (l/m²) is the data value divided by 10. I will also check that the dataset has no suspect or missing values.

In [6]:
import pandas as pd
import numpy as np

#::GMG::Loading data for Santander CMT
#       Skip first 20 line dataset description and include header with column names:
#
#       01-06 STAID: Station identifier
#       08-13 SOUID: Source identifier
#       15-22 DATE : Date YYYYMMDD
#       24-28 RR   : Precipitation amount in 0.1 mm
#       30-34 Q_RR : quality code for RR (0='valid'; 1='suspect'; 9='missing')
#
#       FILE FORMAT (MISSING VALUE CODE = -9999)
#
#::GMG::Pandas read_csv
#       https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html
#
#       Skip 20 first rows ...
#       https://stackoverflow.com/questions/20637439/skip-rows-during-csv-import-pandas#
#       https://stackoverflow.com/questions/27325652/python-pandas-read-csv-skip-rows-but-keep-header/27325729#
#
precip_s = pd.read_csv('RR_STAID003922.txt',skiprows=range(0, 20))

In [54]:
#::GMG::Print the basics of tha dataset
#       see: https://www.shanelynn.ie/using-pandas-dataframe-creating-editing-viewing-data-in-python/
#            https://www.quora.com/How-do-I-get-the-number-of-columns-in-a-pandas-dataframe
#  
#display((type(precip_s),' with ', precip_s.size, ' columns and ', precip_s.count+1,' rows.'))
#print (type(precip_s),'If there was a birth every 7 seconds, there would be: ',births,'births')
print(type(precip_s))
print ('Dataset has {0[0]} rows x {0[1]} columns.'.format(precip_s.shape))
#print('{} rows x {} columns'.format(precip_s.shape))
print('::GMG::Messy columns here (i.e. leading spaces and all): \n',precip_s.columns)
#::GMG::There are 'messy' column names with spaces ... bound to trouble here!
#  see:
#    https://medium.com/@chaimgluck1/working-with-pandas-fixing-messy-column-names-42a54a6659cd
#    https://stackoverflow.com/questions/21606987/how-can-i-strip-the-whitespace-from-pandas-dataframe-headers   
#
precip_s.columns = precip_s.columns.str.strip()
print('Now they are proper colmuns: \n',precip_s.columns)

<class 'pandas.core.frame.DataFrame'>
Dataset has 34273 rows x 5 columns.
::GMG::Messy columns here (i.e. leading spaces and all): 
 Index(['STAID', 'SOUID', 'DATE', 'RR', 'Q_RR'], dtype='object')
Now they are proper colmuns: 
 Index(['STAID', 'SOUID', 'DATE', 'RR', 'Q_RR'], dtype='object')


In [58]:
print('Counting data per column: \n', precip_s.count())
print('Column (numpy) data types: \n', precip_s.dtypes)
print('Some statistics on RR series (mm ~ l/m²):')
print((precip_s['RR']/10).describe())

Counting data per column: 
 STAID    34273
SOUID    34273
DATE     34273
RR       34273
Q_RR     34273
dtype: int64
Column (numpy) data types: 
 STAID    int64
SOUID    int64
DATE     int64
RR       int64
Q_RR     int64
dtype: object
Some statistics on RR series (mm ~ l/m²):
count    34273.000000
mean         3.182873
std          7.264932
min          0.000000
25%          0.000000
50%          0.000000
75%          3.000000
max        126.200000
Name: RR, dtype: float64


In [14]:
print("First rows ...")
precip_s.head(20)

First rows ...


,STAID,SOUID,DATE,RR,Q_RR
0,3922,19972,19240101,1,0
1,3922,19972,19240102,25,0
2,3922,19972,19240103,7,0
3,3922,19972,19240104,96,0
4,3922,19972,19240105,0,0
5,3922,19972,19240106,0,0
6,3922,19972,19240107,4,0
7,3922,19972,19240108,15,0
8,3922,19972,19240109,65,0
9,3922,19972,19240110,13,0


In [15]:
print("... last rows.")
precip_s.tail(20)

... last rows.


,STAID,SOUID,DATE,RR,Q_RR
34253,3922,19981,20171012,0,0
34254,3922,19981,20171013,0,0
34255,3922,19981,20171014,0,0
34256,3922,19981,20171015,0,0
34257,3922,19981,20171016,0,0
34258,3922,19981,20171017,0,0
34259,3922,19981,20171018,40,0
34260,3922,19981,20171019,36,0
34261,3922,19981,20171020,8,0
34262,3922,19981,20171021,6,0
